## Comparison between the Least Square and the Weighted Least Square

As we know, the data in computed tomography is affected by Poisson Noise due to the electromagnetic nature of X-rays. The Poisson distribution can be easily approximated by a Gaussian when the mean is high, but it differs from it when the mean is lower. This is the case of Low Dose CT, where the number of photon that reach the detector pixels are less than standard CT. 
Here we briefly summarize the acquisition model:
$$I = I_{0} e^{-Ax}$$
where $I$ is the intensity data collected by the detector, $I_{0}$ is the incoming souce intensity, $A$ is the projection operator of Computed Tomography and $x$ is the object (attenuation coefficients) that we want to reconstruct. When adding the Poisson Noise we have:
$$ B = Poisson (I). $$ 
Most of the standard CT reconstruction strategies work by solving the linear sistem
$$ Y = Ax$$
where $Y = -\log (\frac{B}{I_{0}})$ is the sinogram data (obtained after taking the negative logarithm of $\frac{B}{I_{0}}$). In particular, the least square problem relies on finding $x$ by solving the following minimization problem:
$$ \min_{x} \mathcal{F}(x) := \frac{1}{2} || Y - Ax||_{2}^{2}.$$
Clearly, using this formulation, we are supposing that the distribution of the noise inside $Y$ is a white Gaussian. We know instead that a Poisson distribution on the measurements $B$ is more realistic, but the function to minimize $\mathcal{F}_{P}$ obtained considering $B =Poisson ( I_{0} e^{-Ax})$  is very different from the linear least-square problem for Gaussian Noise and it needs particular algorithms to minimized. 
By considering a quadratic approximation of function $\mathcal{F}_{P}$  the problem takes the following form:
$$ \min_{x} \mathcal{F_{W}} : = \frac{1}{2} || Y - Ax ||_{W}^{2} \, \rightarrow \, \min_{x} \frac{1}{2} \sum_{i} w_{i} (Y-Ax)_{i} $$
where $w_{i} = e^{-Y_{i}} = \frac{B}{I_{0}}$. The main difference between $\mathcal{F}$ and $\mathcal{F}_{W}$ is the weight matrix $W$.
In this notebook we will see the difference between the solution obtained by minimizing $\mathcal{F}$ and $\mathcal{F}_{W}$ for the case of low dose CT, i.e. where the distribution of the noise is very different from a white gaussian, and the level of the noise is high.

In [ ]:
# cil imports
from cil.framework import ImageData, ImageGeometry, DataContainer
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.optimisation.functions import Function
from cil.processors import Slicer, AbsorptionTransmissionConverter, TransmissionAbsorptionConverter

from cil.optimisation.functions import LeastSquares, WeightedL2NormSquared, KullbackLeibler, OperatorCompositionFunction, ZeroFunction 
from cil.optimisation.functions import IndicatorBox, L2NormSquared, TotalVariation, SumFunction, SmoothMixedL21Norm
from cil.optimisation.algorithms import CGLS, SIRT, GD, PDHG, FISTA
from cil.optimisation.operators import GradientOperator

from cil.plugins.astra.operators import ProjectionOperator
from cil.plugins.astra.processors import FBP

#from cil.version import version
#print (version)

from cil.plugins import TomoPhantom
from cil.utilities.quality_measures import mse, mae, psnr
from cil.utilities import dataexample
from cil.utilities.display import show_geometry

# External imports
from myplot import show2D

import time 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy as sp
import logging
import math
logging.basicConfig(level = logging.INFO)

In [ ]:
# set up default colour map for visualisation
cmap = "gray"

In [ ]:
# set the backend for FBP and the ProjectionOperator
device = 'gpu'

In this notebook we will use a classical Shepp-Logan phantom generated with the [TomoPhantom toolbox](https://github.com/dkazanc/TomoPhantom).

In [ ]:
# number of pixels
n_pixels = 256

# Angles
angles = np.linspace(0, 360, 500, endpoint=False, dtype=np.float32)

# set-up fan-beam AcquisitionGeometry
# distance from source to center of rotation
dist_source_center = 6#300.0
# distance from center of rotation to detector
dist_center_detector = 6# 200.0
# physical pixel size
pixel_size_h = 2/n_pixels

# Setup image geometry
ig = ImageGeometry(voxel_num_x=n_pixels, 
                   voxel_num_y=n_pixels, 
                   voxel_size_x=2/n_pixels, 
                   voxel_size_y=2/n_pixels)
# calculate geometrical magnification
mag = (dist_source_center + dist_center_detector) / dist_source_center

ag = AcquisitionGeometry(geom_type='cone',
                           dimension='2D',
                           angles=angles,
                           pixel_num_h=n_pixels,
                           pixel_size_h=pixel_size_h*mag,
                           dist_source_center=dist_source_center,
                           dist_center_detector=dist_center_detector)



# Setup image geometry
ig = ImageGeometry(voxel_num_x=n_pixels, 
                   voxel_num_y=n_pixels, 
                   voxel_size_x=ag.pixel_size_h / mag, 
                   voxel_size_y=ag.pixel_size_h / mag)
show_geometry(ag)
phantom = TomoPhantom.get_ImageData(num_model=1, geometry=ig)
phantom.array[phantom.array<0] = 0

Next, we create our simulated tomographic data by projecting our noiseless phantom to the acquisition space. 
The variables $\texttt{noisy\_counts}$ and $\texttt{sino\_noisy}$ represent $B$ and $Y$ respectively.

In [ ]:
# Create projection operator using Astra-Toolbox.
A = ProjectionOperator(ig, ag, device)

# Create an acqusition data (numerically)
sino = A.direct(phantom)

In [ ]:
# Incident intensity: lower counts will increase the noise
background_counts = 1000 

# Convert the simulated absorption sinogram to transmission values using Lambert-Beer. 
# Use as mean for Poisson data generation.
# Convert back to absorption sinogram.
counts = background_counts * np.exp(-sino.as_array())  
tmp = np.exp(-sino.as_array())
noisy_counts = np.random.poisson(counts)
nonzero = noisy_counts > 0
sino_out = np.zeros_like(sino.as_array())
sino_out[nonzero] = -np.log(noisy_counts[nonzero] / background_counts)

# allocate sino_noisy and fill with noisy data
sino_noisy = ag.allocate()
sino_noisy.fill(sino_out)

psnr_sino = psnr(sino, sino_noisy, data_range = np.max(sino))

# Visualize the true and noisy data
show2D([counts, noisy_counts], ['true counts', 'noisy counts'], \
       cmap=cmap, num_cols=2, size=(15,10), origin='upper-left')
## Visualize the true and noisy data
show2D([sino, sino_noisy], ['true sino', "noisy sino, psnr = %5.3f" % (psnr_sino)], \
       cmap=cmap, num_cols=2, size=(15,10), origin='upper-left')

# Reconstructing the noisy data using LS TV and WLS TV  (with FISTA)
In addition to the least-square or weighted least square term, we considered a Total Variation Regularizer.
So, for the LS-TV problem, we want to minimize the following:
$$ ||Y-Ax||_{2}^{2} + \alpha \, TV(x) $$
while for the WLS-TV problem we minimize:
$$ ||Y-Ax||_{W}^{2} + \alpha \, TV(x) $$
where $\alpha$ is the regularization parameter that balances the two terms

In [ ]:
#Definition of the fidelity term
LS = LeastSquares(A, sino_noisy)

# Definition of the regularization parameter
alpha_LS = 1

#Initialize quantities
x0 = ig.allocate(0.0)

# Defining the regularization term with the new alpha
alpha1_TV = alpha_LS*TotalVariation()

# Setting up FISTA
myFISTA_LS_TV = FISTA(f=LS, 
                g=alpha1_TV, 
                x_init=x0 ,
                max_iteration=1000)
# Run FISTA
myFISTA_LS_TV.run(1000, verbose=0)
recon_ls_tv = myFISTA_LS_TV.solution
psnr_ls_tv  = psnr(phantom,recon_ls_tv, data_range = np.max(phantom))

In [ ]:
#Definition of the WLS fidelity term
weights = noisy_counts/background_counts
c = DataContainer(weights)
WLS = LeastSquares(A, sino_noisy, 1,c)

# Definition of the regularization parameter
alpha_WLS = 1

#Initialize quantities
x0 = ig.allocate(0.0)

# Defining the regularization term with the new alpha
alpha2_TV = alpha_WLS*TotalVariation()

# Setting up FISTA
myFISTA_WLS_TV = FISTA(f=WLS, 
                g=alpha2_TV, 
                x_init=x0 ,
                max_iteration=1000)
# Run FISTA
myFISTA_WLS_TV.run(1000, verbose=0)
recon_wls_tv = myFISTA_WLS_TV.solution
psnr_wls_tv  = psnr(phantom,recon_wls_tv, data_range = np.max(phantom))

In [ ]:
linenumy = 128  # Line to show in the plot
plt.figure(figsize=(20,10))
plt.plot(phantom.get_slice(horizontal_y=linenumy).as_array(),':k',label="phantom")
plt.plot(recon_ls_tv.get_slice(horizontal_y=linenumy).as_array(),label="LS TV")
plt.plot(recon_wls_tv.get_slice(horizontal_y=linenumy).as_array(),label="WLS TV")
plt.legend(fontsize=20)
plt.title("Reconstruction line horizontal=%3.0f , I0 = %7.f" % (linenumy,background_counts))

In [ ]:
show2D([phantom, recon_ls_tv, recon_wls_tv ], ["phantom, I0 = %7.0f" % (background_counts), "LS TV, alpha=%7.6f, psnr= %5.3f" % (alpha_LS,psnr_ls_tv),\
    "WLS TV alpha=%7.6f, psnr= %5.3f" % (alpha_WLS,psnr_wls_tv)], \
       cmap=cmap, fix_range =(0,1), num_cols=3, size=(15,10), origin='upper-left')

# Attention: the following code needs a lot of time to run
As one can notice from the previous code, the solution strongly depends on the choice of the regularization parameter $\alpha$, whose best value may be different for the two cases considered. For this reason, we can seach ( a posteriori) the best parameter for each strategy, according to the highest PSNR.

In [ ]:
# Selection of the best regularization parametr using LS TV - FISTA
alpha_min = 0.00001
alpha_max = 0.001
alpha_n   = 50
alphas    = np.linspace(alpha_min, alpha_max, alpha_n) 

# Initialize quantities
psnr_ls_tv_alpha = np.zeros_like(alphas)
max_psnr = 0

# Run the loop over the different values of alpha
for i in range(alpha_n):
    alpha = alphas[i]
    # Defining the regularization term with the new alpha
    alphaTV = alpha*TotalVariation()
    # Setting up FISTA
    myFISTATV = FISTA(f=LS, 
                  g=alphaTV, 
                  x_init=x0 ,
                  max_iteration=1000)
    # Run FISTA
    myFISTATV.run(1000, verbose=0)
    recon_ls_tv_all_alphas = myFISTATV.solution
    psnr_ls_tv_alpha[i] = psnr(phantom,recon_ls_tv_all_alphas, data_range = np.max(phantom))
    
    # plot the reconstruction 
    show2D([recon_ls_tv_all_alphas], ["LS TV alpha=%7.6f, psnr = %7.5f" % (alpha,psnr_ls_tv_alpha[i])], \
       cmap=cmap,fix_range=(0,1), size=(10,10), origin='upper-left')
        
    # print the value of alpha and the obtained psnr of the reconstruction
    print("alpha=%7.6f, psnr= %5.3f" % (alpha,psnr_ls_tv_alpha[i]))
    
    # Save the best reconstruction
    if psnr_ls_tv_alpha[i]>max_psnr:
        max_psnr   = psnr_ls_tv_alpha[i]
        best_recon = recon_ls_tv_all_alphas
        best_alpha = alpha

In [ ]:
# Selection of the best regularization parametr using WLS TV - FISTA
alpha_min = 0.00001
alpha_max = 0.001
alpha_n   = 50
alphas    = np.linspace(alpha_min, alpha_max, alpha_n) 

#Initialize quantities
psnr_wls_tv_alpha = np.zeros_like(alphas)
w_max_psnr = 0

# Run the loop over the different values of alpha
for i in range(alpha_n):
    alpha = alphas[i]
    # Defining the regularization term with the new alpha
    alphaTV = alpha*TotalVariation()
    # Setting up FISTA
    myFISTAWTV = FISTA(f=WLS, 
                  g=alphaTV, 
                  x_init=x0 ,
                  max_iteration=1000)
    # Run FISTA
    myFISTAWTV.run(1000, verbose=0)
    recon_wls_tv_all_alphas = myFISTAWTV.solution
    psnr_wls_tv_alpha[i] = psnr(phantom,recon_wls_tv_all_alphas, data_range = np.max(phantom))

    # plot the reconstruction 
    show2D([recon_wls_tv_all_alphas], ["WLS TV alpha=%7.6f, psnr = %7.5f" % (alpha,psnr_wls_tv_alpha[i])], \
       cmap=cmap,fix_range=(0,1), size=(10,10), origin='upper-left')
        
    # print the value of alpha and the obtained psnr of the reconstruction
    print("alpha=%7.6f, psnr= %5.3f" % (alpha,psnr_wls_tv_alpha[i]))
    
    # Save the best reconstruction
    if psnr_wls_tv_alpha[i]>w_max_psnr:
        w_max_psnr   = psnr_wls_tv_alpha[i]
        w_best_recon = recon_wls_tv_all_alphas
        w_best_alpha = alpha

In [ ]:
recon_wls_tv_fista = w_best_recon
psnr_wls_tv_fista  = w_max_psnr
alpha_wls_tv_fista = w_best_alpha

recon_ls_tv_fista = best_recon
psnr_ls_tv_fista  = max_psnr
alpha_ls_tv_fista = best_alpha

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(phantom.get_slice(horizontal_y=linenumy).as_array(),':k',label="phantom")
plt.plot(recon_ls_tv_fista.get_slice(horizontal_y=linenumy).as_array(),label="LS TV")
plt.plot(recon_wls_tv_fista.get_slice(horizontal_y=linenumy).as_array(),label="WLS TV")
plt.legend(fontsize=20)
plt.title("Reconstruction line horizontal best alphas =%3.0f , I0 = %7.f" % (linenumy,background_counts))

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(phantom.get_slice(horizontal_x=linenumy).as_array(),':k',label="phantom")
plt.plot(recon_ls_tv_fista.get_slice(horizontal_x=linenumy).as_array(),label="LS TV")
plt.plot(recon_wls_tv_fista.get_slice(horizontal_x=linenumy).as_array(),label="WLS TV")
plt.legend(fontsize=20)
plt.title("Reconstruction line vertical best alphas =%3.0f , I0 = %7.f" % (linenumy,background_counts))

In [ ]:
show2D([phantom, recon_ls_tv_fista,recon_wls_tv_fista ], ["phantom, I0 = %7.0f" % (background_counts), "FBP, psnr= %5.3f" % (psnr_fbp),\
    "LS TV FISTA alpha=%7.6f, psnr= %5.3f" % (alpha_ls_tv_fista,psnr_ls_tv_fista),\
    "WLS TV FISTA alpha=%7.6f, psnr= %5.3f" % (alpha_wls_tv_fista,psnr_wls_tv_fista)], \
       cmap=cmap, fix_range =(0,1), num_cols=3, size=(15,10), origin='upper-left')

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(alphas,psnr_ls_tv_alpha,label="LS TV")
plt.plot(alphas,psnr_wls_tv_alpha,label="wLS TV")
plt.plot(alpha_ls_tv_fista, psnr_ls_tv_fista, '*r')
plt.plot(alpha_wls_tv_fista, psnr_wls_tv_fista, '*r')
plt.legend(fontsize=20)
plt.title("PSNR for different values of alpha, I0 = %7.f" % (background_counts))